<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/rtofs_aws_download_plot_ssh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RTOFS nowcast (`n`) 2d files from AWS s3

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd

import glob as glob
from PIL import Image

import matplotlib.pyplot as plt
#%matplotlib inline

In [ ]:
def plot_rtofs_ssh(ds, xMin, xMax, yMin, yMax, vName='ssh'):
  # form date string
  yyyymmdd = ds.MT.values[0].astype("str")
  dStr = yyyymmdd.split('T')[0]+'T'+yyyymmdd.split('T')[1].split(':')[0]
  print(dStr)

  fig=plt.figure(figsize=(8, 6), dpi=180)
  ax=fig.add_subplot(111)

  im1=ds[vName].sel(X=slice(xMin, xMax), Y=slice(yMin, yMax)).plot(ax=ax, vmin=-.5, vmax=.5, cmap="gist_ncar", add_colorbar=False)

  im2=ds[vName].sel(X=slice(xMin, xMax), Y=slice(yMin, yMax)).squeeze().plot.contour(ax=ax, vmin=-.5, vmax=.5, cmap="gray")

  cbar=plt.colorbar(im1, ax=ax)
  if vName == 'ssh':
    cbar.set_label("Sea Surface Height (m)")

  ax.set_title("{}".format(dStr))
  ax.set_xlabel("")
  ax.set_ylabel("")

  figName= vName + '_' + dStr + '.png'
  plt.savefig(figName)
  plt.close()

## User inputs

In [ ]:
start_date, end_date = ["20240929", "20241015"]
hours = np.arange(0, 25, 3)

# Plot region bounds in silly x and y!
xMin, xMax = [2300, 2700]
yMin, yMax = [1700, 1900]

### Set fixed parameters

In [ ]:
data_dates = pd.date_range(start=start_date, end=end_date)
url_base = "https://noaa-nws-rtofs-pds.s3.amazonaws.com/rtofs."
fPref = "rtofs_glo_2ds_n"
fSuff = "_diag.nc"

## Download files, plot, save figure and delete downloaded file

In [ ]:
for dd in data_dates:
      print("Downloading data for:\t", dd.strftime("%Y%m%d"))
      for hr in hours:
        url = url_base + dd.strftime("%Y%m%d") + "/" +\
        fPref + str(hr).zfill(3)+ fSuff
        #fName = fPref + str(hr).zfill(3)+ fSuff
        fName_save = fPref + dd.strftime("_%Y%m%d_") + str(hr).zfill(3)+ fSuff
        #print(url, fName_save)

        !wget $url -O $fName_save
        ds = xr.open_dataset(fName_save)
        plot_rtofs_ssh(ds, xMin, xMax, yMin, yMax)
        !rm $fName_save

In [ ]:
!mkdir -p figs
!mv *.png figs/

## Make a gif animation from png files

In [ ]:
png_files_path = 'figs/'
png_files = sorted( glob.glob( png_files_path + '/' + 'ssh_*.png'))
#print(fNames)

DUR, LOOP = [150, 0] # Duration in milliseconds, infinite loop

images = []
for filename in png_files:
  im = Image.open(filename)
  images.append(im)

# save as a gif
fOut = 'rtofs_ssh_' + start_date + '_' + end_date + '.gif'
images[0].save(fOut,save_all=True,
               append_images=images[0:],
               optimize=False,
               duration=DUR, # Duration in milliseconds
               loop=LOOP) # infinite loop

print('\nSaved:\t{}\n'.format(fOut))